# 第３章：正規表現

Wikipediaの記事を以下のフォーマットで書き出したファイル[jawiki-country.json.gz](./jawiki-country.json.gz)がある．

- 1行に1記事の情報がJSON形式で格納される
- 各行には記事名が”title”キーに，記事本文が”text”キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される
- ファイル全体はgzipで圧縮される

以下の処理を行うプログラムを作成せよ．

## 20. JSONデータの読み込み

Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [49]:
import json
import gzip
import os

src_path = "./jawiki-country.json.gz"
trg_path = "./jawiki-country.json"

# gzipファイルの解凍
if(not os.path.isfile(trg_path)):
    with gzip.open(src_path, mode="rb") as zip_f:
        content = zip_f.read()
        with open(trg_path, mode="wb") as f:
            f.write(content)


# jsonファイルの読み込み
with open(trg_path, 'r') as f:
    wiki_data = [json.loads(line) for line in f]

# タイトルがイギリスのtextを抽出し、ファイルに出力
UK_data = [data['text'] for data in wiki_data if data['title'] == "イギリス"]
UK_data = UK_data[0] if len(UK_data) else None
with open("wiki_UK.txt", "w", encoding='utf-8') as f:
    f.write(UK_data)

## 21. カテゴリ名を含む行を抽出

記事中でカテゴリ名を宣言している行を抽出せよ．

In [23]:
# 記事データの読み込み
with open("./wiki_UK.txt", "r") as f:
    lines = f.read().rstrip().split("\n")

# 正規表現でカテゴリ名を抽出
from re import match

pattern = r'^\[\[Category:'
categories = []
for line in lines:
    if(match(pattern, line)):
        categories.append(line)

print(*categories, sep="\n")

# 結果をファイルに出力
with open("./wiki_UK_categories.txt", "w") as f:
    f.write("\n".join(categories) + "\n")

[[Category:イギリス|*]]
[[Category:イギリス連邦加盟国]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国|元]]
[[Category:海洋国家]]
[[Category:現存する君主国]]
[[Category:島国]]
[[Category:1801年に成立した国家・領域]]


## 22. カテゴリ名の抽出

記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [38]:
# カテゴリの行を読み込み
with open("./wiki_UK_categories.txt", "r") as f:
    categories = f.read().rstrip().split("\n")
# 正規表現でカテゴリ名を抽出
from re import search

pattern = r':[^(\||\])]+[\] \|]'
names = []
for category in categories:
    name = search(pattern, category).group()
    if(len(name)):
        names.append(name[1:-1])

print(*names, sep="\n")

イギリス
イギリス連邦加盟国
英連邦王国
G8加盟国
欧州連合加盟国
海洋国家
現存する君主国
島国
1801年に成立した国家・領域


## 23. セクション構造

記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [48]:
# 記事データの読み込み
with open("./wiki_UK.txt", "r") as f:
    lines = f.read().rstrip().split("\n")

# 正規表現でカテゴリ名を抽出
from re import match

pattern = r'={2,}[^=]+={2,}'
with open("wiki_UK_sections.txt", "w") as f:
    for line in lines:
        if(match(pattern, line)):
            level = line.count("=")// 2 - 1
            f.write(str(level)+ "\t" + line + "\n")
            print(level, line, sep="\t")

1	==国名==
1	==歴史==
1	==地理==
2	===主要都市===
2	===気候===
1	==政治==
2	===元首===
2	===法===
2	===内政===
2	===地方行政区分===
2	===外交・軍事===
1	==経済==
2	===鉱業===
2	===農業===
2	===貿易===
2	===不動産===
2	===エネルギー政策===
2	===通貨===
2	===企業===
3	====通信====
1	==交通==
2	===道路===
2	===鉄道===
2	===海運===
2	===航空===
1	==科学技術==
1	==国民==
2	===言語===
2	===宗教===
2	===婚姻===
2	===移住===
2	===教育===
2	===医療===
1	==文化==
2	===食文化===
2	===文学===
2	===哲学===
2	===音楽===
3	====ポピュラー音楽====
2	===映画===
2	===コメディ===
2	===国花===
2	===世界遺産===
2	===祝祭日===
2	===スポーツ===
3	====サッカー====
3	====クリケット====
3	====競馬====
3	====モータースポーツ====
3	====野球====
3	==== カーリング ====
3	==== 自転車競技 ====
1	==脚注==
1	==関連項目==
1	==外部リンク==


## 24. ファイル参照の抽出

記事から参照されているメディアファイルをすべて抜き出せ．